In [1]:
import optuna
import pandas as pd
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train = pd.read_csv("/Users/aswathshakthi/PycharmProjects/MLOps/Predictive_maintenance/data/processed/df_train.csv")
df_train

,engine,cycle,setting_1,setting_2,setting_3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735,4
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594,3
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333,2
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640,1


In [3]:
df_test = pd.read_csv("/Users/aswathshakthi/PycharmProjects/MLOps/Predictive_maintenance/data/processed/df_test.csv")
df_test

,engine,cycle,setting_1,setting_2,setting_3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,112
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,112
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,112
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,112
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974,20
13092,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771,20
13093,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051,20
13094,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699,20


In [4]:
df = pd.concat([df_train, df_test],axis=0)
df

,engine,cycle,setting_1,setting_2,setting_3,s1,s2,s3,s4,s5,...,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.0049,0.0000,100.0,518.67,643.24,1599.45,1415.79,14.62,...,2388.00,8213.28,8.4715,0.03,394,2388,100.0,38.65,23.1974,20
13092,100,195,-0.0011,-0.0001,100.0,518.67,643.22,1595.69,1422.05,14.62,...,2388.09,8210.85,8.4512,0.03,395,2388,100.0,38.57,23.2771,20
13093,100,196,-0.0006,-0.0003,100.0,518.67,643.44,1593.15,1406.82,14.62,...,2388.04,8217.24,8.4569,0.03,395,2388,100.0,38.62,23.2051,20
13094,100,197,-0.0038,0.0001,100.0,518.67,643.26,1594.99,1419.36,14.62,...,2388.08,8220.48,8.4711,0.03,395,2388,100.0,38.66,23.2699,20


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['RUL']), df['RUL'], test_size=0.2, random_state=42)


In [6]:
X_train.drop(columns=["engine","cycle","setting_1","setting_2","setting_3","s1","s5","s6","s10","s16","s18","s19"], inplace=True)
X_test.drop(columns=["engine","cycle","setting_1","setting_2","setting_3","s1","s5","s6","s10","s16","s18","s19"], inplace=True)

In [7]:
# Normalisation

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and test data
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

### Model

In [8]:
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    C = trial.suggest_loguniform('C', 1e-3, 1e2)
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto', 0.1, 0.01])
    kernel = trial.suggest_categorical('kernel', ['rbf'])
    epsilon = trial.suggest_uniform('epsilon', 0.1, 1.0)
    
    # Initialize and train the SVR model
    svr_model = SVR(C=C, gamma=gamma, kernel=kernel, epsilon=epsilon)
    svr_model.fit(X_train_normalized, y_train)
    
    # Predict on test set
    y_pred = svr_model.predict(X_test_normalized)
    
    # Return the MSE
    return mean_squared_error(y_test, y_pred)

In [9]:
# Create study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

[I 2024-09-06 20:06:18,545] A new study created in memory with name: no-name-3161f3ae-2884-4ae3-bbe9-9c3334e7e7fb
[I 2024-09-06 20:06:48,207] Trial 0 finished with value: 3759.4136626658515 and parameters: {'C': 0.025563317895654594, 'gamma': 'auto', 'kernel': 'rbf', 'epsilon': 0.30301045106503843}. Best is trial 0 with value: 3759.4136626658515.
[I 2024-09-06 20:07:17,956] Trial 1 finished with value: 3897.6544717763795 and parameters: {'C': 0.08759658158327045, 'gamma': 0.01, 'kernel': 'rbf', 'epsilon': 0.3548178423307169}. Best is trial 0 with value: 3759.4136626658515.
[I 2024-09-06 20:07:47,477] Trial 2 finished with value: 2865.130622678836 and parameters: {'C': 40.99114283644305, 'gamma': 'scale', 'kernel': 'rbf', 'epsilon': 0.35926739375916716}. Best is trial 2 with value: 2865.130622678836.
[I 2024-09-06 20:08:16,949] Trial 3 finished with value: 3775.907705986679 and parameters: {'C': 0.023961838398786784, 'gamma': 'auto', 'kernel': 'rbf', 'epsilon': 0.5704090377669966}. Best

In [10]:
# Get the best parameters
print(f'Best parameters: {study.best_params}')
best_model = SVR(**study.best_params)
best_model.fit(X_train_normalized, y_train)


Best parameters: {'C': 40.99114283644305, 'gamma': 'scale', 'kernel': 'rbf', 'epsilon': 0.35926739375916716}


SVR(C=40.99114283644305, epsilon=0.35926739375916716)

In [11]:
# Predict using the best model
y_train_pred = best_model.predict(X_train_normalized)
y_test_pred = best_model.predict(X_test_normalized)

 ### Evaluation

In [12]:
#MSE -train
def mse_tr(y_train, y_train_pred):
    mse = mean_squared_error(y_train, y_train_pred)
    print(f'Mean Squared Error - Train: {mse}')
mse_tr(y_train, y_train_pred)

Mean Squared Error - Train: 2770.4130136119734


In [13]:
# R-squared evaluation - train
def r2_tr(y_train, y_train_pred):
    r_squared = r2_score(y_train, y_train_pred)
    print(f'R-squared - Train: {r_squared}')
r2_tr(y_train, y_train_pred)

R-squared - Train: 0.30343528462678837


In [14]:
#MSE -test
def mse_tt(y_test, y_test_pred):
    mse = mean_squared_error(y_test, y_test_pred)
    print(f'Mean Squared Error - Test: {mse}')
mse_tt(y_test, y_test_pred)

Mean Squared Error - Test: 2865.130622678836


In [15]:
# R-squared evaluation - test
def r2_tt(y_test, y_test_pred):
    r_squared = r2_score(y_test, y_test_pred)
    print(f'R-squared - Test: {r_squared}')
r2_tt(y_test, y_test_pred)

R-squared - Test: 0.29482906500508144
